In [1]:
import os
os.environ["PATH"] += os.pathsep + r"/opt/homebrew/opt/ffmpeg@7/bin"


In [1]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
# from datasets import load_dataset


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3-turbo"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
    return_timestamps=True,
    chunk_length_s=10,
    stride_length_s=2,
)
#
# # dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
# # sample = dataset[0]["audio"]
sample = "../audio/lsy_audio_2023_58s.mp3"
#
result = pipe(sample)
# print(result["text"])
#
print(result)

/Users/yeonsigjang/ideaProject/gpt_agent_2025_book/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: numpy>=1.17,<2.0 is required for a normal functioning of this module, but found numpy==2.3.3.
Try: `pip install transformers -U` or `pip install -e '.[dev]'` if you're working with git main

In [4]:
start_end_text = []

for chunk in result["chunks"]:
  start = chunk["timestamp"][0]
  end = chunk["timestamp"][1]
  text = chunk["text"]
  start_end_text.append([start, end, text])

import pandas as pd
df = pd.DataFrame(start_end_text, columns=["start", "end", "text"])
df.to_csv("lsy_audio_2023_58.csv", index=False, sep="|")
display(df)

,start,end,text
0,0.00,6.30,안녕하세요. 이 강의는 GPT API로 챗봇 만들기 라는 내용을 다루는 강의입니다.
1,7.18,10.00,GPT API에 대해서 생소하신 분들도 있을텐데
2,11.00,17.00,"우리가 잘 알고 있는 ChatGPT, ChatGPT 기능을 이용해서"
3,17.00,21.00,우리가 원하는 프로그램을 어떻게 만드는지에 대해서 이야기할 거예요.
4,21.00,24.00,그래서 이런 강의들이 사실 많이 있습니다.
5,24.00,27.48,그래서 여러 가지들이 있는데 이 강의 특징이라고 한다면
6,27.48,29.58,GPT로 명확한 미션을 달성하는
7,29.58,31.66,챕터 프로그램을 만드는게 사실
8,31.66,34.32,쉽지는 않은데 이걸 어떻게 해서
9,34.32,36.40,구현을 하는지 그리고 그게 왜 필요한지에 대해서
